In [1]:
# Goal = genotype all CRISPR 10xmer haploids from multiplex PCR nextseq data

In [1]:
#import packages
import numpy as np
from numpy import matlib
import pandas as pd
import csv
import pylab as pl
import string
import matplotlib.pyplot as plt
import copy
import scipy as stats
from scipy.stats import linregress
from scipy.stats import multinomial
from scipy.stats import t
from scipy.stats import chi2
from scipy.linalg import hadamard
import os
import sys
import itertools
import time
import random
import argparse
from os import listdir
from os.path import isfile, join
from collections import OrderedDict
import gzip

In [2]:
# define indices, loci, and tables for properly extracting info from "BC"
INDEX_LIST = ['CTCGTA','CATTGG','AAGGTC','GACTAC','TTCACG','ACACTT','CGAAGG','TCTTGC']
indexmap = pd.DataFrame()
indexmap['Index'] = INDEX_LIST
indexmap['plateset'] = [1,2,3,4,5,6,7,8]
LOCUS_LIST = ['BUL2','FAS1','MKT1','NCS2','PMA1','RHO5','SCH9','WHI2','AKL1','RPI1','HSL7','SPT7','FRS1']
letters = ['A-E','B-F','C-G','D-H']
letters14 = ['A','B','C','D']
letters58 = ['E','F','G','H']
lettersall = letters14+letters58
letterslash = ['A/E','B/F','C/G','D/H']

indlet = []
indletright = []
for i in np.arange(len(indexmap)):
    for l in np.arange(len(letters)):
        indlet = indlet + [indexmap.loc[i,'plateset'].astype(str)+letters[l]]
        if i <= 3:
            indletright = indletright + [indexmap.loc[i,'plateset'].astype(str)+letters14[l]]
        else:
            indletright = indletright + [indexmap.loc[i,'plateset'].astype(str)+letters58[l]]

indletdf = pd.DataFrame()
indletdf['plateletterlong'] = indlet
indletdf['plateletter'] = indletright

wellund = []
wellright = []
for r in np.arange(len(lettersall)):
    for c in np.arange(3,11):
        wellright = wellright + [lettersall[r]+str(c)]
        if c < 10:
            wellund = wellund + [lettersall[r]+str(c)+'_']
        else:
            wellund = wellund + [lettersall[r]+str(c)]

wellnamemap = pd.DataFrame()
wellnamemap['prewell'] = wellund
wellnamemap['platewell'] = wellright


In [3]:
# import filenames to work off of (excluding A-E_D6)

filenames = pd.read_csv("CRISPR_10xmer-g_array_v02.csv")


# import all data into a dataframe

allcounts = pd.DataFrame()
allcounts['BC'] = ""
allcounts['Reads'] = ""
allcounts['UMI.Count'] = ""
allcounts['Library'] = ""

for f in np.arange(len(filenames)):
    temptable = pd.read_csv("20201102_CRISPR10xmers_g_output_batch/counts/"+filenames.loc[f,'Library']+"_counts.csv")
    temptable['Library'] = filenames.loc[f,'Library']
    allcounts = allcounts.append(temptable)


# Now we want to add in the A/E-D6 reads from the MiSeq run (20201207)
# import all data into a dataframe
temptable = pd.read_csv("20201207_output_miseq/counts/Rxn_51_S67_R1_001.fastq.gz_counts.csv")
temptable['Library'] = "A-E_D6_S28_R1_001.fastq.gz"
allcounts = allcounts.append(temptable)    
    
allcounts = allcounts.reset_index(drop=True)


In [4]:
# Now time to extract the salient features: allele and well identity

allcounts["Index"] = allcounts.BC.str[:6]
allcounts["Locus"] = allcounts.BC.str[7:11]
allcounts["Allele"] = allcounts.BC.str[12:]

allcounts = pd.merge(allcounts,indexmap,on='Index',how='left')

allcounts['letterset'] = allcounts.Library.str[:3]

allcounts['plateletterlong'] = allcounts['plateset'].astype(str)+allcounts['letterset']

allcounts = pd.merge(allcounts,indletdf,on='plateletterlong',how='left')

allcounts['prewell'] = allcounts.Library.str[4:7]

allcounts = pd.merge(allcounts,wellnamemap,on='prewell',how='left')

allcounts['Well'] = allcounts['plateletter']+'-'+allcounts['platewell']

allcounts = allcounts.drop(columns = ['BC','plateset','letterset','plateletterlong','plateletter','platewell','prewell','Index','UMI.Count','Library'])

In [6]:
# Do a quick check for the number of BUL2 reads associated with each well, since this is the one I'm concerned about
#bul2check = allcounts.loc[(allcounts['Locus'] == 'BUL2')].reset_index(drop=True)
#export_csv = bul2check.to_csv(r'bul2check.csv',index=True,header=True)

In [5]:
locusallelemap = allcounts[['Locus','Allele']].drop_duplicates().reset_index(drop=True)

allcounts_allelelist = allcounts.drop(columns = ['Well','Locus'])
allcounts_allelelist = allcounts_allelelist.groupby((['Allele'])).sum()
allcounts_allelelist = pd.merge(allcounts_allelelist,locusallelemap,on='Allele',how='left')

In [8]:
#export_csv = allcounts_allelelist.to_csv(r'20201102_10xmers-g_allcounts-allelelist.csv',index=True,header=True)

In [6]:
# Import list of alleles. Anything allele that has ≥ 10 reads in at least one well and ≥ 1% freq in at least one well gets
# a name, rest is "na" name, lumped together in later stages
allelenames = pd.read_csv("20201103_allelelist_v4.csv")

In [7]:
# Map allele names to big table
allelenames2 = allelenames.drop(columns=['Locus'])
allcounts = pd.merge(allcounts,allelenames2,on='Allele',how='left')

In [8]:
# make a list of all wells present
well_list = list(OrderedDict.fromkeys(allcounts['Well']))


In [30]:
# instead, import acfreq from csv
#acfreq = pd.read_csv("20201208_acfreq.csv")
#acfreq = acfreq.drop(columns = ['Unnamed: 0'])
#acfreq = acfreq.drop(columns=['Unnamed: 0','Allele name','Locus','Complex'])

# do some terraforming to fold in the updated allele list
#acfreq = pd.merge(acfreq,allelenames,how="left",on="Allele")

In [ ]:
# Get a list on which we can do the freq calculations and filtering

blankslist = pd.read_csv("20201104_blankslist.csv")

acfreq = pd.merge(allcounts,blankslist,on='Well',how='left')

acfreqmod = acfreq.copy(deep=True)
acfreqmod = acfreqmod.loc[(acfreqmod['blank?']=='no')].reset_index(drop=True)
acfreqmod['Well_Allele'] = acfreqmod['Well']+'_'+acfreqmod['Allele']

acfreqmod['ComplexsepBY'] = ""
acfreqmod['pool'] = ""

for i in np.arange(len(acfreqmod)):
    if acfreqmod.loc[i,'Allele name'] == 'BY':
        acfreqmod.at[i,'ComplexsepBY'] = 'BY'
    else:
        acfreqmod.at[i,'ComplexsepBY'] = acfreqmod.loc[i,'Complex']
    numr = float(acfreqmod.loc[i,'Well'][0])
    letr = acfreqmod.loc[i,'Well'][1]
    wellr = acfreqmod.loc[i,'Well'][2:]
    if numr < 5:
        acfreqmod.at[i,'pool'] = letterslash[letters14.index(letr)]+wellr
    else:
        acfreqmod.at[i,'pool'] = letterslash[letters58.index(letr)]+wellr

In [ ]:
# Now we need to apply our chimera recognition / filtering algorithm

# create an empty dataframe to add things to
freqstable = pd.DataFrame()

# Create a comprehensive list of pools
poollist = list(OrderedDict.fromkeys(acfreqmod['pool']))

# The outer loop goes pool by pool.
for p in np.arange(240,241):
#THIS TAKES A WHILE TO RUN
#for p in np.arange(len(poollist)):    
    
    # Isolate all the wells/loci/alleles for a given pool
    subt = acfreqmod.loc[(acfreqmod['pool'] == poollist[p])].reset_index(drop=True)
    
    # Look one locus at a time
    for l in np.arange(9,10):
    #for l in np.arange(len(LOCUS_LIST)):        
        subtl = subt.loc[(subt['Locus'] == LOCUS_LIST[l])].reset_index(drop=True)
        
        # reformat this info to prep for chimera filtration
        s2 = pd.DataFrame()
        
        # add in the well names
        s2['Well'] = list(OrderedDict.fromkeys(subtl['Well']))
        
        # list of unique allele names
        thesealleles = list(OrderedDict.fromkeys(subtl['Allele name']))
        
        # Populate the table
        summer = subtl.groupby(['Well','Allele name'])['Reads'].sum().to_frame().reset_index()
        
        for a in np.arange(len(thesealleles)):
            thisallele = thesealleles[a]
            tab2add = summer.loc[(summer['Allele name'] == thisallele)].drop(columns='Allele name').rename(columns={'Reads':thisallele}).reset_index(drop=True)
            s2 = pd.merge(s2,tab2add,on='Well',how='left')
        
        # Replace NaNs with 0 (since not all wells have all alleles)
        s2 = s2.fillna(0)
        
        # For Alex's pipeline, convert to numpy array without headers, well names
        data = s2.drop(columns=['Well']).to_numpy(copy=True)
        
        # HERE BEGINS ALEX'S CODE!
        
        # Add a pseudocount
        data = data + 1
        #print(data)

        # Get chimera pool frequency
        # As opposed to previously, we now have multiple possibilities. We'll always assume that the first column is psWT, but in reality it doesn't matter.
        chimera_count = []
        chimera_freq = []
        for i in range(data.shape[1]):
            chimera_count.append(np.sum(data[:,i]))

        for i in range(data.shape[1]):
            chimera_freq.append(chimera_count[i] / np.sum(chimera_count))


        # Ok let's go well by well.
        min_pis = 0.5

        max_likelihood = -1 * np.Inf
        for it in range(100):
            test_pi = min_pis + (1-min_pis)/100 * it
            probability = 0

            # Ok so what is the model?
            # We assume that a well is the dominant read
            # That means that the probability for observing the dominant read is always 1 * test_pi + (1-test_pi) * chimera_read_probability
            # For observing the non-dominant read it's 0*test_pi + (1-test_pi) * chimera_read_probability

            for i in range(data.shape[0]):
                index_max = np.argmax(data[i])
                p_arr = []
                for j in range(data.shape[1]):
                    if(j == index_max):
                        # This is the type this well is most likely to be.
                        p_arr.append(test_pi * 1 + (1-test_pi) * chimera_freq[j])
                    else:
                        p_arr.append((1-test_pi) * chimera_freq[j])

                probability = probability + multinomial.logpmf(data[i],n=np.sum(data[i]),p=p_arr)

            if(probability > max_likelihood):
                max_likelihood = probability
                max_test_pi = test_pi



        #print("Non chimeric rate: " + str(max_test_pi))

        f_superarr = []
        for i in range(data.shape[0]):
            index_max = np.argmax(data[i])
            well_reads = np.sum(data[i])
            obs_freq = data[i][0] / well_reads

            # Remember that # We assume that a well is the dominant read
            # That means that the probability for observing the dominant read is always 1 * test_pi + (1-test_pi) * chimera_read_probability
            # For observing the non-dominant read it's 0*test_pi + (1-test_pi) * chimera_read_probability

            # f * max_test_pi + (1-max_test_pi) * chimera_freq[j] = data[i][j]/well_reads
            f_arr = []
            for j in range(data.shape[1]):
                f = (data[i][j] / well_reads - (1-max_test_pi) * chimera_freq[j]) / max_test_pi
                f_arr.append(f)

            # ok is the solution valid?
            params = []
            if(len(np.where(np.array(f_arr) < 0)[0]) > 0 or len(np.where(np.array(f_arr) > 1)[0]) > 0):
                # invalid
                # Now search on the boundaries for some valid parameters
                # We need to set up to n-1 parameters to be equal to zero and find the solutions in those cases.
                max_val = pow(2,data.shape[1])
                best_logL = -np.Inf
                best_logL_params = []
                for j in range(1,max_val-1):
                    mask_string = ("{0:b}".format(j).zfill(data.shape[1]))
                    #print(mask_string)
                    enumerated = [place  for place, letter in enumerate(mask_string) if letter == "1"]
                    #print(enumerated)
                    # Inside enumerated are the positions which will not be constrained to zero
                    # Based on this solution: lambda = max_test_pi * (data[i][0] + data[i][1]) / (max_test_pi + (1-max_test_pi) * (chimera_freq1 + chimera_freq2))
                    lambda_val = max_test_pi * np.sum(data[i][enumerated]) / (max_test_pi + (1-max_test_pi) * np.sum(np.array(chimera_freq)[enumerated]))

                    # and:
                    # (max_test_pi * data[i][0] / lambda - (1-max_test_pi) * chimera_freq1) / max_test_pi = f1

                    f_trials = []
                    for k in range(data.shape[1]):
                        if(k in enumerated):
                            f = round((max_test_pi * data[i][k] / lambda_val - (1-max_test_pi) * chimera_freq[k]) / max_test_pi,15)
                            f_trials.append(f)
                        else:
                            f_trials.append(0)
                    # ok now test if that worked
                    #print(f_trials)

                    if(len(np.where(np.array(f_trials) < 0)[0]) == 0 and len(np.where(np.array(f_trials) > 1)[0]) == 0):
                        # valid!
                        #print(f_trials)
                        # Obtain the likelihood of the data!
                        p_arr = np.array(f_trials) * max_test_pi + (1-max_test_pi) * np.array(chimera_freq)
                        #print(p_arr)
                        log_p = multinomial.logpmf(data[i],n=np.sum(data[i]),p=p_arr)
                        #print(log_p)

                        if(log_p > best_logL):
                            best_logL = log_p
                            best_logL_params = f_trials

                #print(best_logL_params)
                f_superarr.append(best_logL_params) 
            else:
                #print(f_arr)   
                f_superarr.append(f_arr)
        
        
        truefreqtable = pd.DataFrame(f_superarr)
        
        # Before tacking on the truefreqtable to s2, want to add some fields.
        
        # First, get the raw frequencies for comparison.
        len2use = len(s2.columns)
        readsums = s2.iloc[:,1:len2use].sum(axis=1)
        for a in np.arange(len(thesealleles)):
            thisallele = thesealleles[a]
            s2[thisallele+'-rfreq'] = s2[thisallele]/readsums
        
        # Next, add the "true frequencies" from the chimera consideration process
        truefreqtable.columns = [s + '-pcfreq' for s in thesealleles]
        s2 = pd.concat([s2,truefreqtable],axis=1)
        
        # Calculate out the inferred reads
        for a in np.arange(len(thesealleles)):
            thisallele = thesealleles[a]
            s2[thisallele+'-pcreads'] = readsums * s2[thisallele+'-pcfreq']
        
        # Let's re-re-format this table in order to make it somewhat tractable.
        s3 = pd.DataFrame()
        
        for a in np.arange(len(thesealleles)):
            thisallele = thesealleles[a]
            temp = s2[['Well',thisallele,thisallele+'-rfreq',thisallele+'-pcfreq',thisallele+'-pcreads']]
            temp = temp.rename(columns={thisallele:'Reads',thisallele+'-rfreq':'Rawfreq',thisallele+'-pcfreq':'pcfreq',thisallele+'-pcreads':'pcReads'})
            temp['Allele name'] = thisallele
            temp['freq-in-pool'] = s2[thisallele].sum()/s2.iloc[:,1:len2use].sum(axis=0).sum()
            s3 = s3.append(temp)
        
        s3['Non-chimeric rate'] = max_test_pi
        s3['Chimeric rate'] = 1 - max_test_pi
        s3['Locus'] = LOCUS_LIST[l]
        s3['pool'] = poollist[p]
        
        freqstable = freqstable.append(s3)
    print(str(p))

freqstable = freqstable.reset_index(drop=True)


#export_csv = freqstable.to_csv(r'20210105_freqstable.csv',index=True,header=True)




In [10]:
# import the freqstable in case don't want to run all this code again!
freqstable = pd.read_csv("20210105_freqstable.csv")
freqstable = freqstable.drop(columns=['Unnamed: 0'])

In [63]:
# Now it's time to apply some thresholds, getting the information in the right format for genotyping
# First, need to map the "complexes" back on.
# To do this, need to create a new field, "Locus-Allele name"
# Also want to collapse the na alleles!
naind1 = allelenames.loc[(allelenames['Allele name'] == 'na')].index.values.tolist()
allelenames.at[naind1,'Allele'] = 'na'
allelenames = allelenames.drop_duplicates().reset_index(drop=True)
allelenames['L-A'] = allelenames['Locus']+'-'+allelenames['Allele name']

freqstable['L-A'] = freqstable['Locus']+'-'+freqstable['Allele name']

# Drop some columns to enable less annoying mapping
allelenames2 = allelenames.drop(columns=['Locus','Allele name'])
ft2 = pd.merge(freqstable,allelenames2,on='L-A',how='left')

In [13]:
# Now get in format for proper genotyping

gt = pd.DataFrame()

new_WL = list(OrderedDict.fromkeys(ft2['Well']))
c_list = ['WT','Mut','Other','na']

gt['Well'] = ""

for l in np.arange(len(LOCUS_LIST)):
    gt[LOCUS_LIST[l]+'-totalreads'] = ''
    for c in np.arange(len(c_list)):
        gt[LOCUS_LIST[l]+'-'+c_list[c]+'%'] = ""
    gt[LOCUS_LIST[l]+'-maxcomplex'] = ''
    gt[LOCUS_LIST[l]+'-maxcomplex%'] = ''

#for w in np.arange(0,1):
for w in np.arange(len(new_WL)):
    gt.at[w,'Well'] = new_WL[w]
    wt = ft2.loc[(ft2['Well'] == new_WL[w])].reset_index(drop=True)
    #for l in np.arange(0,1):
    for l in np.arange(len(LOCUS_LIST)):
        wlt = wt.loc[(wt['Locus'] == LOCUS_LIST[l])].reset_index(drop=True)
        wlts = wlt.groupby('Complex').sum().reset_index()
        gt.at[w,LOCUS_LIST[l]+'-totalreads'] = wlts['Reads'].sum()
        
        cmax = ''
        cpmax = 0
        for c in np.arange(len(c_list)):
            if len(wlts.loc[(wlts['Complex'] == c_list[c])]) == 0:
                gt.at[w,LOCUS_LIST[l]+'-'+c_list[c]+'%'] = 0
            else:
                cp = wlts.loc[(wlts['Complex'] == c_list[c])].reset_index(drop=True).loc[0,'pcfreq']
                gt.at[w,LOCUS_LIST[l]+'-'+c_list[c]+'%'] = cp
                
                if cp > cpmax:
                    cpmax = cp
                    cmax = c_list[c]
        
        gt.at[w,LOCUS_LIST[l]+'-maxcomplex'] = cmax
        gt.at[w,LOCUS_LIST[l]+'-maxcomplex%'] = cpmax

In [14]:
# let's now set a threshold on mixednesss
gt2 = gt.copy(deep=True)

for l in np.arange(len(LOCUS_LIST)):
    gt2[LOCUS_LIST[l]+'_threshterm'] = ''
    
for w in np.arange(len(gt2)):
    for l in np.arange(len(LOCUS_LIST)):
        wl_maxp = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex%']
        # this is the new way I'm trying that subtracts out na %
        wl_maxc = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex']
        wl_nap = gt2.loc[w,LOCUS_LIST[l]+'-na%']
        
        if wl_maxc == 'na':
            gt2.at[w,LOCUS_LIST[l]+'_threshterm'] = 1 - wl_maxp
        
        else:
            gt2.at[w,LOCUS_LIST[l]+'_threshterm'] = 1 - wl_maxp - wl_nap

In [ ]:
# Now that we have the threshterms, let's figure out the threshold for each locus.
# Note that this likely won't work with FRS1 given its unique properties.

threshtable = pd.DataFrame()
threshtable['Locus'] = LOCUS_LIST
threshtable['threshold'] = ''

#for l in np.arange(0,1):
for l in np.arange(len(LOCUS_LIST)):
    curve = sorted(gt2[LOCUS_LIST[l]+'_threshterm'].values.tolist())

    # Code copied from https://stackoverflow.com/a/37121355
    nPoints = len(curve)
    allCoord = np.vstack((range(nPoints), curve)).T
    np.array([range(nPoints), curve])
    firstPoint = allCoord[0]
    lineVec = allCoord[-1] - allCoord[0]
    lineVecNorm = lineVec / np.sqrt(np.sum(lineVec**2))
    vecFromFirst = allCoord - firstPoint
    scalarProduct = np.sum(vecFromFirst * np.matlib.repmat(lineVecNorm, nPoints, 1), axis=1)
    vecFromFirstParallel = np.outer(scalarProduct, lineVecNorm)
    vecToLine = vecFromFirst - vecFromFirstParallel
    distToLine = np.sqrt(np.sum(vecToLine ** 2, axis=1))
    idxOfBestPoint = np.argmax(distToLine)
    
    # Now take that index and convert it into a % cutoff
    lthresh = curve[idxOfBestPoint]
    threshtable.at[l,'threshold'] = lthresh
    print('threshold = '+str(lthresh))
    print('excludes '+str(len(curve) - idxOfBestPoint)+' wells')
    plt.plot(curve)
    plt.title(LOCUS_LIST[l])
    plt.plot(np.linspace(0,1892),[lthresh]*len(np.linspace(0,1892)),alpha=0.5)
    plt.show()
    print('*********************')




In [16]:
# get genotypes based on the threshold
mthresh = threshtable['threshold'].tolist()
            
for l in np.arange(len(LOCUS_LIST)):
    gt2[LOCUS_LIST[l]+'_g'] = ''
    
for w in np.arange(len(gt2)):
    for l in np.arange(len(LOCUS_LIST)):
        wl_maxp = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex%']
        # this is the new way I'm trying that subtracts out na %
        wl_maxc = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex']
        wl_nap = gt2.loc[w,LOCUS_LIST[l]+'-na%']
        
        if wl_maxc == 'na':
            if 1 - wl_maxp < mthresh[l]:
                gt2.at[w,LOCUS_LIST[l]+'_g'] = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex']
            else:
                #gt2.at[w,LOCUS_LIST[l]+'_g'] = 'mixed @ '+str(mthresh[l])
                gt2.at[w,LOCUS_LIST[l]+'_g'] = 'mixed'
        else:
            if 1 - wl_maxp - wl_nap < mthresh[l]:
                gt2.at[w,LOCUS_LIST[l]+'_g'] = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex']
            # this is the old way I tried
            #if 1 - wl_maxp < mthresh:
            #    gt2.at[w,LOCUS_LIST[l]+'_g'] = gt2.loc[w,LOCUS_LIST[l]+'-maxcomplex']
            else:
                #gt2.at[w,LOCUS_LIST[l]+'_g'] = 'mixed @ '+str(mthresh[l])
                gt2.at[w,LOCUS_LIST[l]+'_g'] = 'mixed'
#export_csv = gt2.to_csv(r'20210106_gt2.csv',index=True,header=True)

In [17]:
# We have another trove of single-locus non-pooled sequencing data from our Dec 7 2020 MiSeq run
# Let's use that to supplement our assessment from the main body of data.

# Previously, I've parsed the fastq files into counts files. That's very good.
# I need to import these counts files and assign them to well names and synthesize into a table
# for quick recall and mapping.

# import filenames to work off of

filenames = pd.read_csv("CRISPR_10xmer-g_array_singlelociguys.csv")

# import all data into a dataframe

singlecounts = pd.DataFrame()
singlecounts['BC'] = ""
singlecounts['Reads'] = ""
singlecounts['UMI.Count'] = ""
singlecounts['Library'] = ""
singlecounts['Well'] = ""
singlecounts['Locus'] = ""

for f in np.arange(len(filenames)):
    temptable = pd.read_csv("20201207_output_miseq/counts/"+filenames.loc[f,'Library']+"_counts.csv")
    temptable['Library'] = filenames.loc[f,'Library']
    temptable['Well'] = filenames.loc[f,'Well']
    temptable['Locus'] = filenames.loc[f,'Locus']
    singlecounts = singlecounts.append(temptable)
    
singlecounts = singlecounts.reset_index(drop=True)

# Parse the barcode name for locus
singlecounts["parsedLocus"] = singlecounts.BC.str[7:11]
singlecounts["Allele"] = singlecounts.BC.str[12:]

# Delete all those rows where the parsed locus name matches the supposed locus
singlecounts['locusmatch?'] = singlecounts['Locus'] == singlecounts['parsedLocus']
singlecounts = singlecounts.loc[(singlecounts['locusmatch?'] == True)].reset_index(drop=True)

# Delete useless fields
singlecounts = singlecounts.drop(columns=['BC','UMI.Count','Library','parsedLocus','locusmatch?'])

# Re-import allelelist to re-obtain the na alleles
allelenames = pd.read_csv("20201103_allelelist_v4.csv")
an = allelenames.drop(columns = ['Locus'])

# Map the allele list to singlecounts
singlecounts = pd.merge(singlecounts,an,on='Allele',how='left')

# add in frequencies for each well-locus
scwl = sorted(list(OrderedDict.fromkeys(singlecounts['Well'])))
scll = sorted(list(OrderedDict.fromkeys(singlecounts['Locus'])))

sc2 = pd.DataFrame()

for w in np.arange(len(scwl)):
    for l in np.arange(len(scll)):
        tt = singlecounts.loc[(singlecounts['Well'] == scwl[w])&(singlecounts['Locus'] == scll[l])].reset_index(drop=True)
        ttreadsum = tt['Reads'].sum()
        tt['singlelocus-freq'] = tt['Reads']/ttreadsum
        sc2 = sc2.append(tt)

# Now we want to get things in the same format as we have with the pooled genotypes, such
# that we can merge.

# Now get in format for proper genotyping

sc3 = pd.DataFrame()

sc3['Well'] = ""

for l in np.arange(len(scll)):
    sc3[scll[l]+'-singlelocus-totalreads'] = ''
    for c in np.arange(len(c_list)):
        sc3[scll[l]+'-singlelocus-'+c_list[c]+'%'] = ""
    sc3[scll[l]+'-singlelocus-maxcomplex'] = ''
    sc3[scll[l]+'-singlelocus-maxcomplex%'] = ''

#for w in np.arange(0,1):
for w in np.arange(len(scwl)):
    sc3.at[w,'Well'] = scwl[w]
    wt = sc2.loc[(sc2['Well'] == scwl[w])].reset_index(drop=True)
    #for l in np.arange(0,1):
    for l in np.arange(len(scll)):
        wlt = wt.loc[(wt['Locus'] == scll[l])].reset_index(drop=True)
        wlts = wlt.groupby('Complex').sum().reset_index()
        sc3.at[w,scll[l]+'-singlelocus-totalreads'] = wlts['Reads'].sum()
        
        cmax = ''
        cpmax = 0
        for c in np.arange(len(c_list)):
            if len(wlts.loc[(wlts['Complex'] == c_list[c])]) == 0:
                sc3.at[w,scll[l]+'-singlelocus-'+c_list[c]+'%'] = 0
            else:
                cp = wlts.loc[(wlts['Complex'] == c_list[c])].reset_index(drop=True).loc[0,'singlelocus-freq']
                sc3.at[w,scll[l]+'-singlelocus-'+c_list[c]+'%'] = cp
                
                if cp > cpmax:
                    cpmax = cp
                    cmax = c_list[c]
        
        sc3.at[w,scll[l]+'-singlelocus-maxcomplex'] = cmax
        sc3.at[w,scll[l]+'-singlelocus-maxcomplex%'] = cpmax

        
# Do the threshold-based fields
for l in np.arange(len(scll)):
    sc3[scll[l]+'-singlelocus_threshterm'] = ''
    
for w in np.arange(len(sc3)):
    for l in np.arange(len(scll)):
        wl_maxp = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex%']
        if wl_maxp != 0:
            wl_maxc = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex']
            wl_nap = sc3.loc[w,scll[l]+'-singlelocus-na%']

            if wl_maxc == 'na':
                sc3.at[w,scll[l]+'-singlelocus_threshterm'] = 1 - wl_maxp

            else:
                sc3.at[w,scll[l]+'-singlelocus_threshterm'] = 1 - wl_maxp - wl_nap

for l in np.arange(len(scll)):
    sc3[scll[l]+'-singlelocus_g'] = ''

mthresh = threshtable.loc[(threshtable['Locus'].isin(scll))].reset_index(drop=True)['threshold'].tolist()

for w in np.arange(len(sc3)):
    for l in np.arange(len(scll)):
        wl_maxp = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex%']
        if wl_maxp != 0:
            wl_maxc = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex']
            wl_nap = sc3.loc[w,scll[l]+'-singlelocus-na%']

            if wl_maxc == 'na':
                if 1 - wl_maxp < mthresh[l]:
                    sc3.at[w,scll[l]+'-singlelocus_g'] = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex']
                else:
                    sc3.at[w,scll[l]+'-singlelocus_g'] = 'mixed'
            else:
                if 1 - wl_maxp - wl_nap < mthresh[l]:
                    sc3.at[w,scll[l]+'-singlelocus_g'] = sc3.loc[w,scll[l]+'-singlelocus-maxcomplex']
                else:
                    sc3.at[w,scll[l]+'-singlelocus_g'] = 'mixed'

In [18]:
# map the single locus data onto the bigger data table
gt3 = pd.merge(gt2,sc3,on='Well',how='left')
for l in np.arange(len(scll)):
    gt3[scll[l]+"matches single?"] = ""
    for w in np.arange(len(gt3)):
        if gt3.loc[w,scll[l]+'-singlelocus-maxcomplex'] in c_list:
            gt3.at[w,scll[l]+"matches single?"] = gt3.loc[w,scll[l]+'_g'] == gt3.loc[w,scll[l]+'-singlelocus_g']

# We're gonna go ahead and assume the single-locus data is definitive over the pool data for a given locus
# So that means we're gonna create new fields for updated genotypes, replacing pool genotypes with
# single locus genotypes where possible.

for l in np.arange(len(LOCUS_LIST)):
    gt3[LOCUS_LIST[l]+'_g-update'] = gt3[LOCUS_LIST[l]+'_g']
    if LOCUS_LIST[l] in scll:
        temp = gt3.loc[(gt3[LOCUS_LIST[l]+"matches single?"] == False)]
        tempind = temp.index.values.tolist()
        for i in np.arange(len(tempind)):
            gt3.at[tempind[i],LOCUS_LIST[l]+'_g-update'] = temp.loc[tempind[i],LOCUS_LIST[l]+'-singlelocus_g']

# Now we want these in binary format
for l in np.arange(len(LOCUS_LIST)):
    gt3[LOCUS_LIST[l]+'_g-update_bin'] = ""
    for w in np.arange(len(gt3)):
        if gt3.loc[w,LOCUS_LIST[l]+'_g-update'] == 'WT':
            gt3.at[w,LOCUS_LIST[l]+'_g-update_bin'] = '0'
        elif gt3.loc[w,LOCUS_LIST[l]+'_g-update'] == 'Mut':
            gt3.at[w,LOCUS_LIST[l]+'_g-update_bin'] = '1'
        else:
            gt3.at[w,LOCUS_LIST[l]+'_g-update_bin'] = '2'

# And we want to concatenate to form the full genotypes for the 10xmer and 3 extra loci
gcollist = []
for l in np.arange(len(LOCUS_LIST)):
    gcollist = gcollist + [LOCUS_LIST[l]+'_g-update_bin']
gt3['full_g_bin_13'] = gt3[gcollist].agg(''.join, axis=1)
gt3['full_g_bin_10'] = gt3[gcollist[:10]].agg(''.join, axis=1)

# Map on the expected genotypes
expectations = pd.read_csv('20201104_10xmer-g_expectations.csv').applymap(str)
gt4 = pd.merge(gt3,expectations,on='Well',how='left')

gexplist = []
for l in np.arange(len(LOCUS_LIST)):
    gexplist = gexplist + [LOCUS_LIST[l]+'_g_exp']

gt4['full_g_bin_10_exp'] = gt4[gexplist[:10]].agg(''.join,axis=1)

# Look for matches
gt4['full_g_match?'] = gt4['full_g_bin_10'] == gt4['full_g_bin_10_exp']

#export_csv = gt4.to_csv(r'20200107_gt4.csv',index=True,header=True)

In [ ]:
# Analyze ratios of loci within wells as a possible way to check for aneuploidies

fig, axes = plt.subplots(nrows=13, ncols=13, sharex='col', sharey='row',figsize=(12,12))

for l1 in np.arange(len(LOCUS_LIST)):
    for l2 in np.arange(len(LOCUS_LIST)):
        axes[l1][l2].scatter(gt4[LOCUS_LIST[l2]+'-totalreads'],
                             gt4[LOCUS_LIST[l1]+'-totalreads'],
                             s=0.5)
        axes[l1][l2].set_xlabel(LOCUS_LIST[l2])
        axes[l1][l2].set_ylabel(LOCUS_LIST[l1])

for ax in axes.flat:
    ax.label_outer()
    

delax = []
for i in np.arange(len(LOCUS_LIST)):
    for j in np.arange(len(LOCUS_LIST)):
        if j >= i:
            delax = delax + [[i,j]]
for pair in np.arange(len(delax)):
    fig.delaxes(axes[delax[pair][0]][delax[pair][1]])
#plt.savefig('20210118_readratios_v01.jpg',bbox_inches='tight',dpi=3000)

In [37]:
# Now I want to create a csv mapping barcodes and genotypes to each other for Alex to use in his pipeline for
# fitness, epistasis estimation

# Import the bc well map
bcwellmap = pd.read_csv('CRISPR-10xmer-barcodes-wells-map.csv')

# Create a version of gt4 that is just well names and genotypes at the 13 loci and the match? column
gformapping = gt4[['Well','full_g_match?']].copy(deep=True)
for l in np.arange(len(LOCUS_LIST)):
    gformapping[LOCUS_LIST[l]+'_g-update_bin'] = gt4[LOCUS_LIST[l]+'_g-update_bin']

gformapping = gformapping.copy(deep=True)
gformapping = gformapping.loc[(gformapping['full_g_match?'] == True)].reset_index(drop=True)

bcwellgmap = pd.merge(gformapping,bcwellmap,on='Well',how='left')
bcwellgmap = bcwellgmap.drop(columns=['Well','full_g_match?'])

#export_csv = bcwellgmap.to_csv(r'20210112_10xmer-bc-genotype-map.csv',index=False,header=True)

In [ ]:
# This takes about an hour to run, watch out!
#acfreq = pd.DataFrame()

#for w in np.arange(len(well_list)):
#    for l in np.arange(len(LOCUS_LIST)):
        temptab = allcounts.loc[(allcounts['Well'] == well_list[w])&(allcounts['Locus'] == LOCUS_LIST[l])]
        temptab['freq'] = temptab['Reads']/sum(temptab['Reads'])
        acfreq = acfreq.append(temptab)
    print(well_list[w])

acfreq = acfreq.reset_index(drop=True)

export_csv = acfreq.to_csv(r'20201208_acfreq.csv',index=True,header=True)

In [73]:
# want to get things in the right format now

# create a shell dataframe
genotypes = pd.DataFrame()
genotypes['Well'] = ''

#define alleles of interest, complexes of interest
aoi = ['psWT','Mut','BY']
coi = ['WT','Mut','na']

for l in np.arange(len(LOCUS_LIST)):
    genotypes[LOCUS_LIST[l]+'_totalreads'] = ''
    genotypes[LOCUS_LIST[l]+'_topallele'] = ''
    genotypes[LOCUS_LIST[l]+'_topallelename'] = ''
    genotypes[LOCUS_LIST[l]+'_topfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_topcount'] = ''
    genotypes[LOCUS_LIST[l]+'_psWTfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_psWTcount'] = ''
    genotypes[LOCUS_LIST[l]+'_Mutfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_Mutcount'] = ''
    genotypes[LOCUS_LIST[l]+'_BYfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_BYcount'] = ''
    genotypes[LOCUS_LIST[l]+'_WTcomplexfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_WTcomplexcount'] = ''
    genotypes[LOCUS_LIST[l]+'_Mutcomplexfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_Mutcomplexcount'] = ''
    genotypes[LOCUS_LIST[l]+'_nacomplexfreq'] = ''
    genotypes[LOCUS_LIST[l]+'_nacomplexcount'] = ''

    #genotypes[LOCUS_LIST[l]+'_2ndallele'] = ''
    #genotypes[LOCUS_LIST[l]+'_2ndfreq'] = ''
    #genotypes[LOCUS_LIST[l]+'_2ndcount'] = ''
    #genotypes[LOCUS_LIST[l]+'_errfreq'] = ''

    
# look at each locus and parse the info
for w in np.arange(len(well_list)):
    welltable = acfreq.loc[(acfreq['Well'] == well_list[w])].reset_index(drop=True)
    rowlist = [well_list[w]]
    for l in np.arange(len(LOCUS_LIST)):
        subtable = welltable.loc[(welltable['Locus'] == LOCUS_LIST[l])].sort_values(by=['freq'],ascending=False).reset_index(drop=True)
        # get the total reads
        rowlist = rowlist + [subtable['Reads'].sum()]
        # get the "topallele" stats in gear
        if len(subtable) > 0:
            topa = subtable.loc[0,'Allele']
            topan = subtable.loc[0,'Allele name']
            topf = subtable.loc[0,'freq']
            topr = subtable.loc[0,'Reads']
            rowlist = rowlist + [topa,topan,topf,topr]
        else:
            rowlist = rowlist + [np.nan,np.nan,np.nan,np.nan]
        # get the stats for the alleles of interest
        for a in np.arange(len(aoi)):
            aoitable = subtable.loc[(subtable['Allele name'] == aoi[a])].reset_index(drop=True)
            if len(aoitable) > 0:
                rowlist = rowlist + [aoitable.loc[0,'freq'],aoitable.loc[0,'Reads']]
            else:
                rowlist = rowlist + [np.nan,np.nan]
        # get the stats for the complexes of interest
        ctable = subtable.groupby((['Complex'])).sum()
        for c in np.arange(len(coi)):
            if len(subtable.loc[(subtable['Complex'] == coi[c])]) > 0:
                rowlist = rowlist + [ctable.loc[coi[c],'freq'],ctable.loc[coi[c],'Reads']]
            else:
                rowlist = rowlist + [np.nan,np.nan]
        
    genotypes = genotypes.append(pd.DataFrame([rowlist],columns=list(genotypes)),ignore_index=True)

In [ ]:
# add some useful columns to genotypes table
blankslist = pd.read_csv("20201104_blankslist.csv")
expectations = pd.read_csv('20201104_10xmer-g_expectations.csv')

genotypes = pd.merge(genotypes,blankslist,on='Well',how='left')
genotypes = pd.merge(genotypes,expectations,on='Well',how='left')

In [75]:
# add a genotype guess set of columns for ease, all in one place
for l in np.arange(len(LOCUS_LIST)):
    genotypes[LOCUS_LIST[l]+'_g'] = genotypes[LOCUS_LIST[l]+'_topallelename']

In [76]:
# Now let's export and play with genotypes
export_csv = genotypes.to_csv(r'20201115_genotypes.csv',index=True,header=True)

In [36]:
# Now I want to remove suspected chimeric reads.

# This will require some initial field additions to the acfreq table.
# First, I want to add a field that allows us to see not just the "Complex" but also the "ComplexsepBY".
# Then, add a field that signifies the pool to which each well belongs.
# Then, add a field saying whether a well is blank or not, and DROP blank wells.

acfreqmod = acfreq.copy(deep=True)
acfreqmod['ComplexsepBY'] = ""
acfreqmod['pool'] = ""

for i in np.arange(len(acfreqmod)):
    if acfreqmod.loc[i,'Allele name'] == 'BY':
        acfreqmod.at[i,'ComplexsepBY'] = 'BY'
    else:
        acfreqmod.at[i,'ComplexsepBY'] = acfreqmod.loc[i,'Complex']
    numr = float(acfreqmod.loc[i,'Well'][0])
    letr = acfreqmod.loc[i,'Well'][1]
    wellr = acfreqmod.loc[i,'Well'][2:]
    if numr < 5:
        acfreqmod.at[i,'pool'] = letterslash[letters14.index(letr)]+wellr
    else:
        acfreqmod.at[i,'pool'] = letterslash[letters58.index(letr)]+wellr

acfreqmod = pd.merge(acfreqmod,blankslist,on='Well',how='left')
acfreqmod = acfreqmod.loc[(acfreqmod['blank?']=='no')].reset_index(drop=True)
acfreqmod['Well_Allele'] = acfreqmod['Well']+'_'+acfreqmod['Allele']

In [34]:
#export_csv = acfreqmod.to_csv(r'20201219_acfreqmod.csv',index=True,header=True)

In [54]:
# For testing our read removal algorithm, we create test sets of different pools

opptab = pd.DataFrame.from_dict({'ComplexsepBY': ['WT','Mut'], 'Oppcomp': ['Mut','WT']})

# Create a comprehensive list of pools
poollist = list(OrderedDict.fromkeys(acfreqmod['pool']))

# The outer loop goes pool by pool.
for p in np.arange(244):
#for p in np.arange(len(poollist)):
    
    # Isolate all the wells/loci/alleles for a given pool
    subt = acfreqmod.loc[(acfreqmod['pool'] == poollist[p])].reset_index(drop=True)
    
    # Look one locus at a time
    for l in np.arange(9,10):
    #for l in np.arange(len(LOCUS_LIST)):
        
        subtl = subt.loc[(subt['Locus'] == LOCUS_LIST[l])].reset_index(drop=True)
        
        # Create a new table ("pie") to house frequencies of each allele at that locus
        pie = subtl.drop(columns=['Reads','Well','freq','blank?']).drop_duplicates().reset_index(drop=True)
        readr = subtl.groupby('Allele')['Reads'].sum().to_frame()
        pie = pd.merge(pie,readr,on='Allele',how='left')
        pie['poolfreq_allele'] = pie['Reads']/sum(pie['Reads'])
        
        piec = pie.groupby('ComplexsepBY')['poolfreq_allele'].sum().to_frame().reset_index()
        
        # Figure out the "Complex" composition of each well at this locus
        # Need to format complextable with complexes absent from the wells as placeholders still.
        subtl_wells = sorted(list(OrderedDict.fromkeys(subtl['Well'])))
        complexes = ['WT','Mut','BY','Other','na']
        completer = pd.DataFrame()
        completer['Well'] = ""
        completer['ComplexsepBY'] = ""
        for w in np.arange(len(subtl_wells)):
            subcompleter = pd.DataFrame()
            subcompleter['ComplexsepBY'] = complexes
            subcompleter['Well'] = subtl_wells[w]
            completer = completer.append(subcompleter)
            
        complextable = subtl.groupby(['Well','ComplexsepBY']).sum().reset_index()
        complextable = pd.merge(completer,complextable,on=['Well','ComplexsepBY'],how='left')
        
        nanind = complextable[pd.isnull(complextable['Reads'])].index.values
        complextable.at[nanind,'Reads'] = 0
        complextable.at[nanind,'freq'] = 0
        
        # Create a table where we store the info on the leading (max_freq) complex for each well
        lc = subtl.groupby(['Well','ComplexsepBY']).sum().reset_index()
        lc = lc[lc.groupby(['Well'])['freq'].transform(max) == lc['freq']]
        
        # For the purpose of calculating the % of chimeric reads, use only those for which lead complex is WT or Mut
        lc_forav = lc.loc[(lc['ComplexsepBY'] == 'WT')].append(lc.loc[(lc['ComplexsepBY'] == 'Mut')])
        
        # Create a field to hold the opposite complex name
        lc_forav = pd.merge(lc_forav,opptab,on='ComplexsepBY',how='left')
        
        # Map the frequencies of the Oppcomp onto the lc_forav table
        lc_forav = pd.merge(lc_forav,complextable,left_on=['Well','Oppcomp'],right_on=['Well','ComplexsepBY'],how='left',
                            suffixes=('','_opp')).drop(columns=['Oppcomp'])



In [ ]:
# Now time for the big loop to make the read removal happen.

# This is a df for later...
opptab = pd.DataFrame.from_dict({'ComplexsepBY': ['WT','Mut'], 'Oppcomp': ['Mut','WT']})

# And this is the threshold at which we will say that a well has too many reads (by %) that are "actually" bad.
# Here, "actually bad" means something slightly different than "chimeric." It means "having an undesirable
# phenotype." So there may be things we don't want to use for the purposes of the algorithm below (e.g.,
# a well that's supposed to be psWT that's actually mostly BY). The divergence between this definition of "bad"
# and the deeper definition of "chimeric" buried in the algorithm might result in some unsavory inconsistencies
# or loops, so keep an eye out.
malthresh = 0.05

# Create a comprehensive list of pools
poollist = list(OrderedDict.fromkeys(acfreqmod['pool']))

# Create a place to put all information generated by the algorithm that we might want to refer back to.
# This will contain more info than needed I expect, historical info for each iteration included,
# so make sure to filter on that as appropriate during analysis.
piestore = pd.DataFrame()
freqstore = pd.DataFrame()

# The outer loop goes pool by pool.
for p in np.arange(1):
#for p in np.arange(len(poollist)):
    
    # Isolate all the wells/loci/alleles for a given pool
    subt = acfreqmod.loc[(acfreqmod['pool'] == poollist[p])].reset_index(drop=True)
    
    # Look one locus at a time
    for l in np.arange(8,9):
    #for l in np.arange(len(LOCUS_LIST)):
        
        loopflag = 0
        itcount = 0
        
        subtl = subt.loc[(subt['Locus'] == LOCUS_LIST[l])].reset_index(drop=True)
        
        # Create a new table ("pie") to house frequencies of each allele at that locus
        pie = subtl.drop(columns=['Reads','Well','freq','blank?']).drop_duplicates().reset_index(drop=True)
        readr = subtl.groupby('Allele')['Reads'].sum().to_frame()
        pie = pd.merge(pie,readr,on='Allele',how='left')
        pie['poolfreq_allele'] = pie['Reads']/sum(pie['Reads'])
        
        piestore = piestore.append(pie)
        
        piec = pie.groupby('ComplexsepBY')['poolfreq_allele'].sum().to_frame().reset_index()
        
        # Figure out the "Complex" composition of each well at this locus
        # Need to format complextable with complexes absent from the wells as placeholders still.
        subtl_wells = sorted(list(OrderedDict.fromkeys(subtl['Well'])))
        complexes = ['WT','Mut','BY','Other','na']
        completer = pd.DataFrame()
        completer['Well'] = ""
        completer['ComplexsepBY'] = ""
        for w in np.arange(len(subtl_wells)):
            subcompleter = pd.DataFrame()
            subcompleter['ComplexsepBY'] = complexes
            subcompleter['Well'] = subtl_wells[w]
            completer = completer.append(subcompleter)
            
        complextable = subtl.groupby(['Well','ComplexsepBY']).sum().reset_index()
        complextable = pd.merge(completer,complextable,on=['Well','ComplexsepBY'],how='left')
        
        nanind = complextable[pd.isnull(complextable['Reads'])].index.values
        complextable.at[nanind,'Reads'] = 0
        complextable.at[nanind,'freq'] = 0
        
        # Create a table where we store the info on the leading (max_freq) complex for each well
        lc = subtl.groupby(['Well','ComplexsepBY']).sum().reset_index()
        lc = lc[lc.groupby(['Well'])['freq'].transform(max) == lc['freq']]
        
        # For the purpose of calculating the % of chimeric reads, use only those for which lead complex is WT or Mut
        lc_forav = lc.loc[(lc['ComplexsepBY'] == 'WT')].append(lc.loc[(lc['ComplexsepBY'] == 'Mut')])
        
        # Create a field to hold the opposite complex name
        lc_forav = pd.merge(lc_forav,opptab,on='ComplexsepBY',how='left')
        
        # Map the frequencies of the Oppcomp onto the lc_forav table
        lc_forav = pd.merge(lc_forav,complextable,left_on=['Well','Oppcomp'],right_on=['Well','ComplexsepBY'],how='left',
                            suffixes=('','_opp')).drop(columns=['Oppcomp'])
        
        # Replace NaNs with 0
        #lc_forav[pd.isnull(lc_forav['Reads_opp'])] = 0
        
        # Based on the pool-wide "pie" of alleles, estimate fraction of chimeric reads in each well
        lc_forav['chim_freq'] = ""
        for w in np.arange(len(lc_forav)):
            if len(piec.loc[(piec['ComplexsepBY'] == lc_forav.loc[w,'ComplexsepBY_opp']),'poolfreq_allele']) > 0:
                lc_forav.at[w,'chim_freq'] = lc_forav.loc[w,'freq_opp'] / piec.loc[(piec['ComplexsepBY'] == lc_forav.loc[w,'ComplexsepBY_opp']),'poolfreq_allele'].reset_index(drop=True)[0]
            else:
                lc_forav.at[w,'chim_freq'] = 0
            
        
        # Now take the average of the chim_freq across the wells to treat as the "true" chimera frequency across the pool
        # IMPORTANT: Changed this to median to be robust to weird small # / small # stuff. Might need more stable solution later.
        chim_avg = lc_forav['chim_freq'].median()
        
        # Now we want to look at each well in turn
        judgmentbank = []
        freqotherlist = []
        freqstoreint = pd.DataFrame()
        #for w in np.arange(5,6):
        for w in np.arange(len(subtl_wells)):
            subtlw = subtl.loc[(subtl['Well'] == subtl_wells[w])].reset_index(drop=True)
            
            # Get total reads at that locus for that well
            totreadslw = subtlw['Reads'].sum()
            
            # Using the average est. chimera freq and pie, calculate how many reads that is for each allele
            pie['toremove_'+subtl_wells[w]] = chim_avg * totreadslw * pie['poolfreq_allele']
            piew = pie[['Allele','poolfreq_allele','toremove_'+subtl_wells[w]]]
            piew = piew.rename(columns={'toremove_'+subtl_wells[w]:'toremove'})
            
            # Merge that new DataFrame with the well-locus subtable
            subtlw = pd.merge(subtlw,piew,on='Allele',how='left')
            
            # Subtract these putatively chimeric reads from the raw reads for each allele,
            # being sure not to go negative.
            subtlw['Reads_adj'] = subtlw['Reads'] - subtlw['toremove']
            nanind2 = subtlw[subtlw['Reads_adj'] < 0].index.values
            subtlw.at[nanind2,'Reads_adj'] = 0
            
            # Calculate the old and new % of reads at each locus in the well
            subtlw['wellfreq_allele'] = subtlw['Reads']/sum(subtlw['Reads'])
            subtlw['wellfreq_allele_adj'] = subtlw['Reads_adj']/sum(subtlw['Reads_adj'])
            
            # for storage purposes
            subtlw['iteration'] = itcount
            
            # Evaluate whether this given well is "bad" and store that info
            lcw = subtlw.groupby(['Complex']).sum().reset_index()
            lcw = lcw.sort_values(by=['wellfreq_allele_adj'],ascending=False).reset_index(drop=True)
            freqother = 1 - lcw.loc[0,'wellfreq_allele_adj']
            if freqother > malthresh:
                judgmentbank = judgmentbank + ['bad (mixed)']
                subtlw['judgment'] = 'bad (mixed)'
                subtlw['judgment_freqother'] = freqother
            else:
                judgmentbank = judgmentbank + ['good']
                subtlw['judgment'] = 'good'
                subtlw['judgment_freqother'] = freqother
            freqotherlist = freqotherlist + [freqother]
            freqstoreint = freqstoreint.append(subtlw)
                
        # Ask if any are bad (mixed). If not, proceed to the next pool-locus.
        # If so, repeat the process for this pool-locus, but remove the one (worst) problem well from the pool
        # in the part of the algorithm that figures out the average chimerism.
        
        if judgmentbank.count('bad (mixed)') == 0:
            loopflag = 1
            freqstoreint['final_it?'] = "yes"
            freqstore = freqstore.append(freqstoreint)
            
        while loopflag == 0:
            
            freqstoreint['final_it?'] = "no"
            freqstore = freqstore.append(freqstoreint)
            
            itcount = itcount+1
            
            dfh = pd.DataFrame()
            dfh['Well'] = subtl_wells
            dfh['judge'] = judgmentbank
            dfh['freq-not-lead-complex'] = freqotherlist
            dfhbad = dfh.loc[(dfh['judge'] == 'bad (mixed)')].sort_values(by=['freq-not-lead-complex'],ascending=False).reset_index(drop=True)
            subtl_wells = subtl_wells.remove(dfhbad.loc[0,'Well'])
             
            pie = pie[['Locus','Allele','Allele name','Complex','ComplexsepBY','pool','Reads','poolfreq_allele']]
            
            lc_forav = lc_forav.loc[(lc_forav['Well'] != dfhbad.loc[0,'Well'])].reset_index(drop=True)

            # Now take the average of the chim_freq across the wells to treat as the "true" chimera frequency across the pool
            # IMPORTANT: As above, using median for now, may want to change approach in future.
            chim_avg = lc_forav['chim_freq'].median()

            # Now we want to look at each well in turn
            judgmentbank = []
            freqotherlist = []
            freqstoreint = pd.DataFrame()
            #for w in np.arange(1,2):
            for w in np.arange(len(subtl_wells)):
                subtlw = subtl.loc[(subtl['Well'] == subtl_wells[w])].reset_index(drop=True)

                # Get total reads at that locus for that well
                totreadslw = subtlw['Reads'].sum()

                # Using the average est. chimera freq and pie, calculate how many reads that is for each allele
                pie['toremove_'+subtl_wells[w]] = chim_avg * totreadslw * pie['poolfreq_allele']
                piew = pie[['Allele','poolfreq_allele','toremove_'+subtl_wells[w]]]
                piew = piew.rename(columns={'toremove_'+subtl_wells[w]:'toremove'})

                # Merge that new DataFrame with the well-locus subtable
                subtlw = pd.merge(subtlw,piew,on='Allele',how='left')

                # Subtract these putatively chimeric reads from the raw reads for each allele,
                # being sure not to go negative.
                subtlw['Reads_adj'] = subtlw['Reads'] - subtlw['toremove']
                nanind2 = subtlw[subtlw['Reads_adj'] < 0].index.values
                subtlw.at[nanind2,'Reads_adj'] = 0

                # Calculate the old and new % of reads at each locus in the well
                subtlw['wellfreq_allele'] = subtlw['Reads']/sum(subtlw['Reads'])
                subtlw['wellfreq_allele_adj'] = subtlw['Reads_adj']/sum(subtlw['Reads_adj'])

                # for storage purposes
                subtlw['iteration'] = itcount

                # Evaluate whether this given well is "bad" and store that info
                lcw = subtlw.groupby(['Complex']).sum().reset_index()
                lcw = lcw.sort_values(by=['wellfreq_allele_adj'],ascending=False).reset_index(drop=True)
                freqother = 1 - lcw.loc[0,'wellfreq_allele_adj']
                if freqother > malthresh:
                    judgmentbank = judgmentbank + ['bad (mixed)']
                    subtlw['judgment'] = 'bad (mixed)'
                    subtlw['judgment_freqother'] = freqother
                else:
                    judgmentbank = judgmentbank + ['good']
                    subtlw['judgment'] = 'good'
                    subtlw['judgment_freqother'] = freqother
                freqotherlist = freqotherlist + [freqother]
                freqstoreint = freqstoreint.append(subtlw)
                
            if judgmentbank.count('bad (mixed)') == 0:
                freqstoreint['final_it?'] = "yes"
                freqstore = freqstore.append(freqstoreint)
                loopflag = 1
                
            while loopflag == 0:

                freqstoreint['final_it?'] = "no"
                freqstore = freqstore.append(freqstoreint)

                itcount = itcount+1

                dfh = pd.DataFrame()
                dfh['Well'] = subtl_wells
                dfh['judge'] = judgmentbank
                dfh['freq-not-lead-complex'] = freqotherlist
                dfhbad = dfh.loc[(dfh['judge'] == 'bad (mixed)')].sort_values(by=['freq-not-lead-complex'],ascending=False).reset_index(drop=True)
                subtl_wells = subtl_wells.remove(dfhbad.loc[0,'Well'])

                pie = pie[['Locus','Allele','Allele name','Complex','ComplexsepBY','pool','Reads','poolfreq_allele']]

                lc_forav = lc_forav.loc[(lc_forav['Well'] != dfhbad.loc[0,'Well'])].reset_index(drop=True)

                # Now take the average of the chim_freq across the wells to treat as the "true" chimera frequency across the pool
                # IMPORTANT: As above, using median for now, may want to change approach in future.
                chim_avg = lc_forav['chim_freq'].median()

                # Now we want to look at each well in turn
                judgmentbank = []
                freqotherlist = []
                freqstoreint = pd.DataFrame()
                #for w in np.arange(1,2):
                for w in np.arange(len(subtl_wells)):
                    subtlw = subtl.loc[(subtl['Well'] == subtl_wells[w])].reset_index(drop=True)

                    # Get total reads at that locus for that well
                    totreadslw = subtlw['Reads'].sum()

                    # Using the average est. chimera freq and pie, calculate how many reads that is for each allele
                    pie['toremove_'+subtl_wells[w]] = chim_avg * totreadslw * pie['poolfreq_allele']
                    piew = pie[['Allele','poolfreq_allele','toremove_'+subtl_wells[w]]]
                    piew = piew.rename(columns={'toremove_'+subtl_wells[w]:'toremove'})

                    # Merge that new DataFrame with the well-locus subtable
                    subtlw = pd.merge(subtlw,piew,on='Allele',how='left')

                    # Subtract these putatively chimeric reads from the raw reads for each allele,
                    # being sure not to go negative.
                    subtlw['Reads_adj'] = subtlw['Reads'] - subtlw['toremove']
                    nanind2 = subtlw[subtlw['Reads_adj'] < 0].index.values
                    subtlw.at[nanind,'Reads_adj'] = 0

                    # Calculate the old and new % of reads at each locus in the well
                    subtlw['wellfreq_allele'] = subtlw['Reads']/sum(subtlw['Reads'])
                    subtlw['wellfreq_allele_adj'] = subtlw['Reads_adj']/sum(subtlw['Reads_adj'])

                    # for storage purposes
                    subtlw['iteration'] = itcount

                    # Evaluate whether this given well is "bad" and store that info
                    lcw = subtlw.groupby(['Complex']).sum().reset_index()
                    lcw = lcw.sort_values(by=['wellfreq_allele_adj'],ascending=False).reset_index(drop=True)
                    freqother = 1 - lcw.loc[0,'wellfreq_allele_adj']
                    if freqother > malthresh:
                        judgmentbank = judgmentbank + ['bad (mixed)']
                        subtlw['judgment'] = 'bad (mixed)'
                        subtlw['judgment_freqother'] = freqother
                    else:
                        judgmentbank = judgmentbank + ['good']
                        subtlw['judgment'] = 'good'
                        subtlw['judgment_freqother'] = freqother
                    freqotherlist = freqotherlist + [freqother]
                    freqstoreint = freqstoreint.append(subtlw)

                if judgmentbank.count('bad (mixed)') == 0:
                    freqstoreint['final_it?'] = "yes"
                    freqstore = freqstore.append(freqstoreint)
                    loopflag = 1
                

In [ ]:
# I believe the below may all be deprecated, but forget

In [101]:
lc_forav = lc.loc[(lc['ComplexsepBY'] == 'WT')].append(lc.loc[(lc['ComplexsepBY'] == 'Mut')])
        
# Create a field to hold the opposite complex name
lc_forav = pd.merge(lc_forav,opptab,on='ComplexsepBY',how='left')

# Map the frequencies of the Oppcomp onto the lc_forav table
lc_forav = pd.merge(lc_forav,complextable,left_on=['Well','Oppcomp'],right_on=['Well','ComplexsepBY'],how='left',
                    suffixes=('','_opp')).drop(columns=['Oppcomp'])

lc_forav
lc_forav['chim_freq'] = ""
lc_forav.loc[w,'freq_opp']
len(piec.loc[(piec['ComplexsepBY'] == lc_forav.loc[w,'ComplexsepBY_opp']),'poolfreq_allele'])
#for w in np.arange(len(lc_forav)):
#    lc_forav.at[w,'chim_freq'] = lc_forav.loc[w,'freq_opp'] / piec.loc[(piec['ComplexsepBY'] == lc_forav.loc[w,'ComplexsepBY_opp']),'poolfreq_allele'][0]

0

In [ ]:


# before any further read pruning, let's evaluate the chimera test data set

# first, import the chimera test libraries
# import filenames to work off of

filenamesc = pd.read_csv("CRISPR_10xmer-g-followup_array_v01_chim.csv")


# import all data into a dataframe

allcountsc = pd.DataFrame()
allcountsc['BC'] = ""
allcountsc['Reads'] = ""
allcountsc['UMI.Count'] = ""
allcountsc['Library'] = ""

for f in np.arange(len(filenamesc)):
    temptable = pd.read_csv("20201207_output_miseq/counts/"+filenamesc.loc[f,'Library']+"_counts.csv")
    temptable['Library'] = filenamesc.loc[f,'Library']
    allcountsc = allcountsc.append(temptable)
    
allcountsc = allcountsc.reset_index(drop=True)

# Now time to extract the allele identity

allcountsc["Index"] = allcountsc.BC.str[:6]
allcountsc["Locus"] = allcountsc.BC.str[7:11]
allcountsc["Allele"] = allcountsc.BC.str[12:]

# Map on the allele name
allcountsc = pd.merge(allcountsc,allelenames2,on='Allele',how='left')

# Make a column with Library-Index as a proxy for "Well" in the allele freq calculation
allcountsc['Library-Index'] = allcountsc['Library'] + "-" + allcountsc['Index']

# make a list of all the Library-Index pairs
li_list = list(OrderedDict.fromkeys(allcountsc['Library-Index']))

# Calculate allele frequencies
acfreqc = pd.DataFrame()

for li in np.arange(len(li_list)):
    for l in np.arange(len(LOCUS_LIST)):
        temptab = allcountsc.loc[(allcountsc['Library-Index'] == li_list[li])&(allcountsc['Locus'] == LOCUS_LIST[l])]
        temptab['freq'] = temptab['Reads']/sum(temptab['Reads'])
        acfreqc = acfreqc.append(temptab)
    #print(li_list[li])

acfreqc = acfreqc.reset_index(drop=True)

#export_csv = acfreqc.to_csv(r'20201207_acfreqc.csv')

In [ ]:
# I want to get a sense of the balance between alleles for each locus
for l in np.arange(len(LOCUS_LIST)):
    temptab = acfreq.loc[(acfreq['Locus'] == LOCUS_LIST[l])]
    temptab = temptab.drop(columns=['Complex','Locus','freq','Well','Allele'])
    temptab = temptab.groupby((['Allele name'])).sum()
    print(LOCUS_LIST[l])
    print(temptab)
# looks like Mut and psWT alleles predominate, but some loci where significant fraction is one of the inc alleles.

In [ ]:
# Now let's filter the table, screening out alleles that take up less than X% of a given locus's reads in a given well

percthresh = 0.01

acfreq_thresh = acfreq.loc[(acfreq['freq'] >= percthresh)].reset_index(drop=True)

#export_csv = acfreq_thresh.to_csv(r'20201103_acfreq_thresh.csv')

# Now let's pull out the read counts for each well and locus to see what we can see
counttable = acfreq.copy(deep=True).drop(columns=['Allele','Allele name','freq'])
counttable = counttable.groupby(['Well','Locus']).sum()

#export_csv = counttable.to_csv(r'20201103_well-locus_counts.csv')

In [5]:
#checking out the regextable for export (2021.09.12)
regex_table = pd.DataFrame()
regex_table['locus']=['BUL2','FAS1','MKT1','NCS2','PMA1','RHO5','SCH9','WHI2','AKL1','RPI1','HSL7','SPT7','FRS1']
regex_table['left']=['CAACACAA','TACCAGGA','ATAATTGT','ATCCTGAA','TGTTTGTC','GTGTGATA','TCATTTCT','ATTTAATG','CTTGATAT','ATGGAAAG','ATCTGAAT','CAACCAAT','GCCAACCA']
regex_table['right']=['CTAACGTT','TAATTTAG','CATTATGT','CAGAATCT','TAATAGCA','AAAACGTC','GCCATTAA','TTCACCAA','AAGGTAGT','ATTACTAC','GATTTGCC','CATCCCTG','TCAACGGA']
regex_table['bp'] = [20,23,21,20,20,21,21,21,22,20,21,20,20]
export_csv = regex_table.to_csv(r'CRISPRgenotyping_regextable.csv')